In [1]:
#Dependencies
!pip install pgeocode

import pgeocode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from gmap_config import gkey

In [2]:
merged_file = "/Users/ryan/Desktop/bootcamp/group1/Project-1/MergedData.csv"

lst_str_cols = ['Zipcode']

dict_dtypes = {x : 'str'  for x in lst_str_cols}

In [4]:
#read csv and convert all columns to string (because it keeps chopping off)
merged_df = pd.read_csv(merged_file, dtype=dict_dtypes)
merged_df

,Unnamed: 0,Zipcode,state,Total_4_Household Income,Total_4_Asian Amer.,Total_4_Black Amer.,Total_4_Hispanic Amer.,Total_4_White Amer.,Total_4_Other
0,0,07001,34,343075.0,18426.0,22192.0,17039.0,44953.0,3357.0
1,1,07002,34,289847.0,31204.0,35874.0,94537.0,216586.0,31253.0
2,2,07003,34,375154.0,21905.0,46003.0,68466.0,150661.0,16794.0
3,3,07004,34,554234.0,1221.0,0.0,2475.0,35704.0,72.0
4,4,07005,34,538634.0,6511.0,2160.0,7265.0,64701.0,1204.0
...,...,...,...,...,...,...,...,...,...
590,590,08011,34,NaN,0.0,0.0,0.0,107.0,0.0
591,591,07977,34,NaN,54.0,214.0,1001.0,2572.0,298.0
592,592,08038,34,NaN,0.0,0.0,0.0,399.0,0.0
593,593,08073,34,NaN,0.0,0.0,0.0,1094.0,209.0


In [5]:
merged_df = merged_df.drop(["Unnamed: 0"], axis=1)
merged_df

,Zipcode,state,Total_4_Household Income,Total_4_Asian Amer.,Total_4_Black Amer.,Total_4_Hispanic Amer.,Total_4_White Amer.,Total_4_Other
0,07001,34,343075.0,18426.0,22192.0,17039.0,44953.0,3357.0
1,07002,34,289847.0,31204.0,35874.0,94537.0,216586.0,31253.0
2,07003,34,375154.0,21905.0,46003.0,68466.0,150661.0,16794.0
3,07004,34,554234.0,1221.0,0.0,2475.0,35704.0,72.0
4,07005,34,538634.0,6511.0,2160.0,7265.0,64701.0,1204.0
...,...,...,...,...,...,...,...,...
590,08011,34,NaN,0.0,0.0,0.0,107.0,0.0
591,07977,34,NaN,54.0,214.0,1001.0,2572.0,298.0
592,08038,34,NaN,0.0,0.0,0.0,399.0,0.0
593,08073,34,NaN,0.0,0.0,0.0,1094.0,209.0


In [12]:
merged_df = merged_df.rename(columns={"Total_4_Household Income": "Total Race Income",
                                      "Total_4_Asian Amer.": "Asian Income",
                                      "Total_4_Black Amer.": "Black Income",
                                      "Total_4_Hispanic Amer.": "Hispanic Income",
                                      "Total_4_White Amer.": "White Income",
                                      "Total_4_Other": "Other Race Income"
                                     })
merged_df

TypeError: '<' not supported between instances of 'str' and 'int'

In [7]:
lats=[]
longs =[]
nomi = pgeocode.Nominatim('us')
for zips in merged_df['Zipcode']:
    lat = nomi.query_postal_code(zips).latitude
    lon = nomi.query_postal_code(zips).longitude
    lats.append(lat)
    longs.append(lon)
      
merged_df['lats']= lats
merged_df['longs']= longs
merged_df = merged_df[merged_df['lats'].notnull()]
merged_df

,Zipcode,state,Total Race Income,Asian Income,Black Income,Hispanic Income,White Income,Other Race Income,lats,longs
0,07001,34,343075.0,18426.0,22192.0,17039.0,44953.0,3357.0,40.5826,-74.2785
1,07002,34,289847.0,31204.0,35874.0,94537.0,216586.0,31253.0,40.6664,-74.1192
2,07003,34,375154.0,21905.0,46003.0,68466.0,150661.0,16794.0,40.8035,-74.1891
3,07004,34,554234.0,1221.0,0.0,2475.0,35704.0,72.0,40.8822,-74.2960
4,07005,34,538634.0,6511.0,2160.0,7265.0,64701.0,1204.0,40.9115,-74.4140
...,...,...,...,...,...,...,...,...,...,...
590,08011,34,NaN,0.0,0.0,0.0,107.0,0.0,39.9760,-74.7114
591,07977,34,NaN,54.0,214.0,1001.0,2572.0,298.0,40.7079,-74.6541
592,08038,34,NaN,0.0,0.0,0.0,399.0,0.0,39.4716,-75.4902
593,08073,34,NaN,0.0,0.0,0.0,1094.0,209.0,40.0095,-74.8668


In [8]:
# Storing 'Lat' and 'Lng' into  locations 
locations = merged_df[['lats', 'longs']].astype(float)
locations

,lats,longs
0,40.5826,-74.2785
1,40.6664,-74.1192
2,40.8035,-74.1891
3,40.8822,-74.2960
4,40.9115,-74.4140
...,...,...
590,39.9760,-74.7114
591,40.7079,-74.6541
592,39.4716,-75.4902
593,40.0095,-74.8668


In [15]:
# Converting total race income to float and store
total_race_income = merged_df['Total Race Income'].astype(float)

total_race_income = total_race_income.apply(lambda x: x if x > 0 else 0)
total_race_income


0      343075.0
1      289847.0
2      375154.0
3      554234.0
4      538634.0
         ...   
590         0.0
591         0.0
592         0.0
593         0.0
594         0.0
Name: Total Race Income, Length: 595, dtype: float64

In [19]:
# Configure gmaps
gmaps.configure(gkey)
#Define Locations and Weigh
#Create figure
fig = gmaps.figure()
#Create heat layer
fig.add_layer(gmaps.heatmap_layer(locations, weights=total_race_income, dissipating=False, point_radius=0.05))
fig

Figure(layout=FigureLayout(height='420px'))